In [9]:
import leaderbot
import leaderbot as lb
import numpy as np
import matplotlib.pyplot as plt
import texplot
import time
import scipy

In [2]:
data = lb.data.load()

agents = np.array(data['models'])
x = np.array(data['X'])
y = np.array(data['Y'])
n_agents = len(agents)

# Clean empty data
y_sum = np.sum(y, axis=1)
ind_valid = y_sum != 0
x = x[ind_valid]
y = y[ind_valid]

In [3]:
# wins, losses, ties = y[:, 0], y[:, 1], y[:, 2]

# n_wins = np.zeros((n_agents,), dtype=int)
# n_losses = np.zeros((n_agents,), dtype=int)
# n_ties = np.zeros((n_agents,), dtype=int)
# n_games = np.zeros((n_agents,), dtype=int)

# for i in range(n_agents):
#     ind_0 = (x[:, 0] == i)
#     ind_1 = (x[:, 1] == i)
#     n_wins[i] = np.sum(wins[ind_0]) + np.sum(losses[ind_1])
#     n_losses[i] = np.sum(losses[ind_0]) + np.sum(wins[ind_1])
#     n_ties[i] = np.sum(ties[ind_0]) + np.sum(ties[ind_1])

# n_matches = n_wins + n_losses + n_ties
# p_wins = n_wins / n_matches
# p_losses = n_losses / n_matches
# p_ties = n_ties / n_matches

In [9]:
#algs = [davidson, davidson_scaled, davidson_scaled_r, davidson_scaled_rij]
model = lb.DavidsonScaled(data)
model.train()

In [10]:
p_pred = model.infer()

# Evaluate All

In [1]:
import leaderbot as lb
import numpy as np

In [2]:
data = lb.data.load()
training_data, test_data = lb.data.split(data, test_ratio=0.2)

In [3]:
models = [
    lb.models.BradleyTerry(training_data),
    lb.models.RaoKupper(training_data),
    lb.models.Davidson(training_data)
]

In [9]:
models = [
    lb.models.BradleyTerry(training_data),
    lb.models.BradleyTerryScaled(training_data),
    lb.models.BradleyTerryScaledR(training_data),
    # lb.models.BradleyTerryScaledRIJ(training_data),
    lb.models.RaoKupper(training_data),
    lb.models.RaoKupperScaled(training_data),
    lb.models.RaoKupperScaledR(training_data),
    # lb.models.RaoKupperScaledRIJ(dtraining_data),
    lb.models.Davidson(training_data),
    lb.models.DavidsonScaled(training_data),
    lb.models.DavidsonScaledR(training_data),
    # lb.models.DavidsonScaledRIJ(training_data),
]

In [10]:
for model in models:

    name = model.__class__.__name__
    if name.endswith('ScaledRIJ'):
        method = 'L-BFGS-B'
    else:
        method = 'BFGS'

    print(f'training {name:<21} ... ', end='')
    model.train()
    print('done.')

training BradleyTerry          ... done.
training BradleyTerryScaled    ... done.
training BradleyTerryScaledR   ... done.
training RaoKupper             ... done.
training RaoKupperScaled       ... done.
training RaoKupperScaledR      ... done.
training Davidson              ... done.
training DavidsonScaled        ... done.
training DavidsonScaledR       ... done.


In [11]:
metrics = lb.evaluate.model_selection(models, train=False, tie=True, report=True)

+-----------------------+---------+--------+--------+--------+---------+
| model                 | # param | NLL    | CEL    | AIC    | BIC     |
+-----------------------+---------+--------+--------+--------+---------+
| BradleyTerry          |     129 | 0.6544 |    inf | 256.69 | 1020.94 |
| BradleyTerryScaled    |     258 | 0.6542 |    inf | 514.69 | 2043.20 |
| BradleyTerryScaledR   |     259 | 0.6542 |    inf | 516.69 | 2051.12 |
| RaoKupper             |     130 | 1.0080 | 1.0080 | 257.98 | 1028.16 |
| RaoKupperScaled       |     259 | 1.0077 | 1.0077 | 515.98 | 2050.41 |
| RaoKupperScaledR      |     260 | 1.0077 | 1.0077 | 517.98 | 2058.34 |
| Davidson              |     130 | 1.0085 | 1.0085 | 257.98 | 1028.16 |
| DavidsonScaled        |     259 | 1.0083 | 1.0083 | 515.98 | 2050.41 |
| DavidsonScaledR       |     260 | 1.0083 | 1.0083 | 517.98 | 2058.34 |
+-----------------------+---------+--------+--------+--------+---------+


In [12]:
metrics = lb.evaluate.goodness_of_fit(models, train=False, tie=False, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |    Mean Absolute Error     |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          | 10.98  10.98  -----  10.98 | 0.0199 | 0.5687 |
| BradleyTerryScaled    | 10.44  10.44  -----  10.44 | 0.0189 | 0.5409 |
| BradleyTerryScaledR   | 10.42  10.42  -----  10.42 | 0.0188 | 0.5396 |
| RaoKupper             |  8.77   9.10  11.66   9.84 | 0.0331 | 0.9176 |
| RaoKupperScaled       |  8.47   8.55  11.67   9.56 | 0.0322 | 0.8919 |
| RaoKupperScaledR      |  8.40   8.56  11.66   9.54 | 0.0322 | 0.8949 |
| Davidson              |  8.91   9.36  12.40  10.22 | 0.0341 | 0.9445 |
| DavidsonScaled        |  8.75   8.74  12.47   9.99 | 0.0332 | 0.9217 |
| DavidsonScaledR       |  8.73   8.72  12.48   9.98 | 0.0331 | 0.9201 |
+-----------------------+--------------------------

In [13]:
metrics = lb.evaluate.generalization(models, test_data=test_data, train=False, tie=False, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |    Mean Absolute Error     |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          | 10.80  10.80  -----  10.80 | 0.0203 | 0.6100 |
| BradleyTerryScaled    |  9.90   9.90  -----   9.90 | 0.0193 | 0.5840 |
| BradleyTerryScaledR   |  9.88   9.88  -----   9.88 | 0.0194 | 0.5865 |
| RaoKupper             |  8.90   8.40  10.43   9.24 | 0.0339 | 0.9295 |
| RaoKupperScaled       |  8.77   8.47  10.49   9.24 | 0.0332 | 0.9103 |
| RaoKupperScaledR      |  8.91   8.79  10.50   9.40 | 0.0350 | 0.9257 |
| Davidson              |  8.97   8.58  11.06   9.54 | 0.0345 | 0.9478 |
| DavidsonScaled        |  8.83   8.56  11.14   9.51 | 0.0337 | 0.9290 |
| DavidsonScaledR       |  8.83   8.60  11.15   9.53 | 0.0338 | 0.9309 |
+-----------------------+--------------------------